In [46]:
!pip install yfinance sqlalchemy

 Solution to the recent verison of SQLAlchemy:

 - https://levelup.gitconnected.com/how-to-fix-attributeerror-optionengine-object-has-no-attribute-execute-in-pandas-eb635fbb89e4

In [47]:
import pandas as pd 
from sqlalchemy import create_engine, text
import yfinance as yf

In [48]:
engine = create_engine('sqlite:///TEST_DB.db') 

In [49]:
df = yf.download('AAPL', start='2022-01-01', end='2023-01-01')

[*********************100%***********************]  1 of 1 completed


In [50]:
df

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-01-03,177.830002,182.880005,177.710007,182.009995,180.434311,104487900
2022-01-04,182.630005,182.940002,179.119995,179.699997,178.144287,99310400
2022-01-05,179.610001,180.169998,174.639999,174.919998,173.405670,94537600
2022-01-06,172.699997,175.300003,171.639999,172.000000,170.510941,96904000
2022-01-07,172.889999,174.139999,171.029999,172.169998,170.679489,86709100
...,...,...,...,...,...,...
2022-12-23,130.919998,132.419998,129.639999,131.860001,131.477127,63814900
2022-12-27,131.380005,131.410004,128.720001,130.029999,129.652435,69007800
2022-12-28,129.669998,131.029999,125.870003,126.040001,125.674026,85438400


In [51]:
df.to_sql('AAPL', engine)

ValueError: Table 'AAPL' already exists.

In [52]:
query = 'SELECT * FROM AAPL'

with engine.begin() as conn:
  aapl = pd.read_sql_query(sql=text(query), con=conn)

In [53]:
aapl

,Date,Open,High,Low,Close,Adj Close,Volume
0,2022-01-03 00:00:00.000000,177.830002,182.880005,177.710007,182.009995,180.434296,104487900
1,2022-01-04 00:00:00.000000,182.630005,182.940002,179.119995,179.699997,178.144287,99310400
2,2022-01-05 00:00:00.000000,179.610001,180.169998,174.639999,174.919998,173.405670,94537600
3,2022-01-06 00:00:00.000000,172.699997,175.300003,171.639999,172.000000,170.510956,96904000
4,2022-01-07 00:00:00.000000,172.889999,174.139999,171.029999,172.169998,170.679489,86709100
...,...,...,...,...,...,...,...
246,2022-12-23 00:00:00.000000,130.919998,132.419998,129.639999,131.860001,131.477127,63814900
247,2022-12-27 00:00:00.000000,131.380005,131.410004,128.720001,130.029999,129.652435,69007800
248,2022-12-28 00:00:00.000000,129.669998,131.029999,125.870003,126.040001,125.674026,85438400
249,2022-12-29 00:00:00.000000,127.989998,130.479996,127.730003,129.610001,129.233658,75703700


In [54]:
with engine.begin() as conn:
  max_date = pd.read_sql_query(sql=text('SELECT MAX(Date) FROM AAPL'), con=conn).values[0][0]

In [55]:
max_date

'2022-12-30 00:00:00.000000'

In [56]:
df[df.index > max_date].to_sql('AAPL', engine, if_exists='append')

0

In [79]:
# Make the system scalable
def sql_importer(symbol, start='2022-01-01'):
    try:
        with engine.begin() as conn:
            max_date = pd.read_sql(sql=text(f'SELECT MAX(Date) FROM {symbol}'), con=conn).values[0][0]
        print(max_date)
        new_data = yf.download(symbol, start=pd.to_datetime(max_date))
        new_rows = new_data[new_data.index > max_date]
        new_rows.to_sql(symbol, engine, if_exists='append')
        if len(new_rows) == 0:
            print(f"the DB {symbol} doesn't have data to import")
        print(f'{len(new_rows)} new rows impoted to DB')
    except:
        new_data = yf.download(symbol, start=start)
        new_data.to_sql(symbol, engine)
        print(f'New table created for {symbol} with {len(new_data)} rows')

In [73]:
sql_importer('TSLA')

[*********************100%***********************]  1 of 1 completed
New table created for TSLA with 346 rows


In [74]:
stocks = ['GOOG', 'TSLA', 'MMM', 'AAPL']

In [80]:
for stock in stocks:
    sql_importer(stock)

2023-05-18 00:00:00.000000
[*********************100%***********************]  1 of 1 completed
the DB GOOG doesn't have data to import
0 new rows impoted to DB
2023-05-18 00:00:00.000000
[*********************100%***********************]  1 of 1 completed
the DB TSLA doesn't have data to import
0 new rows impoted to DB
2023-05-18 00:00:00.000000
[*********************100%***********************]  1 of 1 completed
the DB MMM doesn't have data to import
0 new rows impoted to DB
2023-05-18 00:00:00.000000
[*********************100%***********************]  1 of 1 completed
the DB AAPL doesn't have data to import
0 new rows impoted to DB
